## הגדרות ומקדים

### ספריות

In [29]:
import os
import sys
import pandas as pd
import geopandas as gpd

In [30]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

### העלת משתנים להרצת הקוד

In [31]:
cwd = os.getcwd()

create_forecast_basic_folder_path = os.path.dirname(cwd)

sys.path.append(create_forecast_basic_folder_path)

## פונקציות

### פונקציות גלובליות

In [32]:
from global_functions import up_load_shp, up_load_df, drop_geo, up_load_gdb

## להריץ תלמידים

In [33]:
%run "students_current_year.ipynb"

## להעלות שכבות

In [34]:
path=r'{}\background_files\EMP_KIBOLET.gdb'.format(create_forecast_basic_folder_path)

EMP_kibolet=up_load_gdb(path,'EMP_kibolet')

In [35]:
EMP_kibolet=EMP_kibolet.fillna(0)

EMP_kibolet['emp_without_palestin']=EMP_kibolet['kayim_emp']-EMP_kibolet['Palestinians']

emp_without_palestin_not_okev=EMP_kibolet['emp_without_palestin'].sum().item()

emp_Education=taz.query('main_secto!="Palestinian"')['emp_Education'].sum().item()

## הוספת סטודנטים לאוכלוסיה ולהתפלגות גילים

In [36]:
student_dorms=up_load_shp(r'{}\background_files\student_dorms.shp'.format(cwd))

In [37]:
#העלה של נתוני אנשים התפלגות גילים שנוצר לפני הקוד הזה
path=r'{}\Intermediates'.format(cwd)
taz_with_pop=up_load_df(path,'taz_with_pop_info')

In [38]:
col_needed=['Taz_num','aprt_20',
 'pop',
 'pop_0',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_5',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up', 'hh_size']

In [39]:
taz=taz.merge(taz_with_pop[col_needed],on='Taz_num',how='left')

In [40]:
taz=taz.set_index('Taz_num')

In [41]:
taz['student_dorms']=gpd.sjoin(taz[['geometry']].reset_index(),student_dorms)[['Taz_num','student_nu']].pivot_table(index='Taz_num',aggfunc='sum')


In [42]:

taz=taz.fillna(0)

taz['pop_without_dorms_yeshiva']=taz['pop']

taz['pop']=taz['pop']+taz['student_dorms']

taz['pop_20_just_from_aprt']=taz['pop_20']

taz['pop_25_just_from_aprt']=taz['pop_25']

taz['pop_20']=taz['pop_20']+taz['student_dorms']*0.6

taz['pop_25']=taz['pop_25']+taz['student_dorms']*0.4

In [43]:
col=['pop_0',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_5',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up',]

taz['pop_check']=round(taz[col].sum(axis=1)-taz['pop'])

taz.loc[taz['main_secto']!="Palestinian"].loc[taz['pop_check']!=0]

,Taz_name,jeru_metro,in_jerusal,main_secto,SCHN_NAME,ENG_NAME_n,zonetype,Muni_Heb,Sug_Muni,CR_PNIM,highBusine,SCHOOLDIST,Urban,EIProp,poly_puma,F3,F2,F1,agri,Indus,Com_hotel,Business,Public,commuting,ieold,IEProp,perScaled,FreeBuffer,PaidBuffer,Rest_EmpBu,searchtime,walktime,cost,Agg_taz_nu,area,CITYCODE1,CITYCODE2,CITYCODE3,CITYCODE4,codeseq,codeseqCon,county,majunivenr,parktot,superZone,Taz1,slope,REGION,yosh,jerusalem_,geometry,SEA1,SEA2,SEA3,UOA1,UOA2,UOA3,ARA1,ARA2,ARA3,student_chardi_not_gov,student_arab_not_gov,TOA1,TOA2,TOA3,sector_for_chinuc,student,Univ_AR,Univ_SE,Univ_UO,emp_uni,univ,yeshiva_dorms_pop_15,yeshiva_dorms_pop_20,yeshiva_dorms_pop_25,yeshiva_dorms_pop_sum,emp_from_Yeshiva_student,Yeshiva,Seminar_dorms_pop_15,Seminar_dorms_pop_20,Seminar_dorms_pop_25,Seminar_dorms_pop_sum,emp_from_Seminar_student,Seminar,kollim_demand,add_from_kollim_demand,student_yeshiva_and_kollim,UO_Hi_Ed,emp_UO_Hi_Ed,UNIVENRORTHFEMALE,UNIVENRORTHMALE,jew,emp_from_student,emp_Education,aprt_20,pop,pop_0,pop_10,pop_15,pop_20,pop_25,pop_30,pop_35,pop_40,pop_45,pop_5,pop_50,pop_55,pop_60,pop_65,pop_70,pop_75up,hh_size,student_dorms,pop_without_dorms_yeshiva,pop_20_just_from_aprt,pop_25_just_from_aprt,pop_check
Taz_num,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


## הוספת תלמידי ישיבה (באזורים חרדיים) אל כמות אוכלוסיה,התפלגות גילים 

In [44]:
taz['pop_15_just_from_aprt']=taz['pop_15']

In [45]:

taz.loc[taz['main_secto']=="U_Orthodox",'pop']=taz['pop']+taz['yeshiva_dorms_pop_sum']

taz.loc[taz['main_secto']=="U_Orthodox",'pop_15']=taz['pop_15']+taz['yeshiva_dorms_pop_15']

taz.loc[taz['main_secto']=="U_Orthodox",'pop_20']=taz['pop_20']+taz['yeshiva_dorms_pop_20']

taz.loc[taz['main_secto']=="U_Orthodox",'pop_25']=taz['pop_25']+taz['yeshiva_dorms_pop_25']

In [46]:
col=['pop_0',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_5',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up',]

taz['pop_check']=round(taz[col].sum(axis=1)-taz['pop'])

taz.loc[taz['main_secto']!="Palestinian"].loc[taz['pop_check']!=0]

,Taz_name,jeru_metro,in_jerusal,main_secto,SCHN_NAME,ENG_NAME_n,zonetype,Muni_Heb,Sug_Muni,CR_PNIM,highBusine,SCHOOLDIST,Urban,EIProp,poly_puma,F3,F2,F1,agri,Indus,Com_hotel,Business,Public,commuting,ieold,IEProp,perScaled,FreeBuffer,PaidBuffer,Rest_EmpBu,searchtime,walktime,cost,Agg_taz_nu,area,CITYCODE1,CITYCODE2,CITYCODE3,CITYCODE4,codeseq,codeseqCon,county,majunivenr,parktot,superZone,Taz1,slope,REGION,yosh,jerusalem_,geometry,SEA1,SEA2,SEA3,UOA1,UOA2,UOA3,ARA1,ARA2,ARA3,student_chardi_not_gov,student_arab_not_gov,TOA1,TOA2,TOA3,sector_for_chinuc,student,Univ_AR,Univ_SE,Univ_UO,emp_uni,univ,yeshiva_dorms_pop_15,yeshiva_dorms_pop_20,yeshiva_dorms_pop_25,yeshiva_dorms_pop_sum,emp_from_Yeshiva_student,Yeshiva,Seminar_dorms_pop_15,Seminar_dorms_pop_20,Seminar_dorms_pop_25,Seminar_dorms_pop_sum,emp_from_Seminar_student,Seminar,kollim_demand,add_from_kollim_demand,student_yeshiva_and_kollim,UO_Hi_Ed,emp_UO_Hi_Ed,UNIVENRORTHFEMALE,UNIVENRORTHMALE,jew,emp_from_student,emp_Education,aprt_20,pop,pop_0,pop_10,pop_15,pop_20,pop_25,pop_30,pop_35,pop_40,pop_45,pop_5,pop_50,pop_55,pop_60,pop_65,pop_70,pop_75up,hh_size,student_dorms,pop_without_dorms_yeshiva,pop_20_just_from_aprt,pop_25_just_from_aprt,pop_check,pop_15_just_from_aprt
Taz_num,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


## כימות מועסקים במרחב צתאל

In [47]:
pre_woman=0.5

pre_man=1-pre_woman

In [48]:
work_age=[ 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_50',
 'pop_55',
 'pop_60']

under_work_age=[ 'pop_15', 'pop_20']

over_work_age=[ 'pop_65', 'pop_70', 'pop_75up']

In [49]:
taz['work_age']=taz[work_age].sum(axis=1)

taz['under_work_age']=taz[under_work_age].sum(axis=1)

taz['over_work_age']=taz[over_work_age].sum(axis=1)

In [50]:
sector='U_Orthodox'

taz.loc[taz['main_secto']==sector,'pop_emp']=taz['work_age']*pre_woman*0.75+taz['work_age']*pre_man*0.55
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['under_work_age']*pre_woman*0.07+taz['under_work_age']*pre_man*0.09
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['over_work_age']*pre_woman*0.05+taz['over_work_age']*pre_man*0.09

sector='Jewish'

taz.loc[taz['main_secto']==sector,'pop_emp']=taz['work_age']*pre_woman*0.9+taz['work_age']*pre_man*0.92
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['under_work_age']*pre_woman*0.20+taz['under_work_age']*pre_man*0.15
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['over_work_age']*pre_woman*0.2+taz['over_work_age']*pre_man*0.15

sector='Arab'

taz.loc[taz['main_secto']==sector,'pop_emp']=taz['work_age']*pre_woman*0.25+taz['work_age']*pre_man*0.7
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['under_work_age']*pre_woman*0.2+taz['under_work_age']*pre_man*0.15
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['over_work_age']*pre_woman*0.05+taz['over_work_age']*pre_man*0.09

sector='arabs_behined_seperation_wall'

taz.loc[taz['main_secto']==sector,'pop_emp']=taz['work_age']*pre_woman*0.25+taz['work_age']*pre_man*0.7
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['under_work_age']*pre_woman*0.2+taz['under_work_age']*pre_man*0.15
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['over_work_age']*pre_woman*0.05+taz['over_work_age']*pre_man*0.09


## חישוב אבטלה

In [51]:
taz['pop_emp_employed']=0

taz.loc[taz['main_secto']=="U_Orthodox",'pop_emp_employed']=taz['pop_emp']*0.97

taz.loc[taz['main_secto']=="Jewish",'pop_emp_employed']=taz['pop_emp']*0.98

taz.loc[taz['jew']==0,'pop_emp_employed']=taz['pop_emp']*0.95

## חישוב יוממות

In [52]:
taz['pop_emp_employed_out_of_jtmt_area']=taz['pop_emp_employed']*taz['commuting']

emp_left_jtmt_area=taz['pop_emp_employed_out_of_jtmt_area'].sum().item()

round(emp_left_jtmt_area,-3)

98000.0

In [53]:
emp_from_jtmt_area=taz['pop_emp_employed'].sum().item()-emp_left_jtmt_area

emp_in_jtmt_area=emp_from_jtmt_area*1.07 #יוממות פנימה מחוץ למרחב

round(emp_in_jtmt_area,-3)

506000.0

In [54]:
emp_okev=emp_in_jtmt_area-emp_Education-emp_without_palestin_not_okev

## פיזור עוקב

In [55]:
EMP_kibolet['geometry_buff']=EMP_kibolet.buffer(250)

EMP_buffer=EMP_kibolet.set_geometry('geometry_buff').query('kayim_emp>0 & effects_okev==1')

EMP_buffer=EMP_buffer.dissolve()[['geometry_buff']]

taz['taz_area']=taz.area

In [56]:
emp_buffer_taz=gpd.overlay(EMP_buffer,taz.reset_index())

emp_buffer_taz['emp_samll_area']=emp_buffer_taz.area

emp_buffer_taz['emp_pre_from_taz']=emp_buffer_taz['emp_samll_area']/emp_buffer_taz['taz_area']

In [57]:
taz_num_no_need_okev=emp_buffer_taz.loc[emp_buffer_taz['emp_pre_from_taz']>0.4].Taz_num.to_list()

In [58]:
taz['okev']=0
taz.loc[(~taz.index.isin(taz_num_no_need_okev))&(taz['pop']>0)&(taz['main_secto']!="Palestinia"),'okev']=1

taz.loc[taz['main_secto']=="arabs_behined_seperation_wall",'okev']=1

## מקדם עוקב

In [59]:
taz['emp_okev']=0

taz.loc[(taz['jew']==0)&(taz['okev']==1),'emp_okev']=taz['aprt_20']*0.3

taz.loc[(taz['main_secto']=="U_Orthodox")&(taz['okev']==1),'emp_okev']=taz['aprt_20']*0.3

taz.loc[(taz['main_secto']=="Jewish")&(taz['okev']==1),'emp_okev']=taz['aprt_20']*0.2


In [60]:

delta_okev_down_from_up=taz.emp_okev.sum().item()-emp_okev
round(delta_okev_down_from_up,-3)

-85000.0

## פיצול שכבת לא עוקב לאזורי תנועה

In [61]:
EMP_kibolet['emp_area']=EMP_kibolet.area

EMP_kibolet['ID']=EMP_kibolet.index

EMP_kibolet_by_taz=gpd.overlay(taz.reset_index()[['Taz_num','taz_area','geometry']],EMP_kibolet)

EMP_kibolet_by_taz['small_area']=EMP_kibolet_by_taz.area

C:\Users\gidon\AppData\Local\Temp\ipykernel_30156\1806380584.py:5: UserWarning: `keep_geom_type=True` in overlay resulted in 287 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  EMP_kibolet_by_taz=gpd.overlay(taz.reset_index()[['Taz_num','taz_area','geometry']],EMP_kibolet)


In [62]:
EMP_kibolet_by_taz['pre_small_area_emp']=EMP_kibolet_by_taz['small_area']/EMP_kibolet_by_taz['emp_area']

EMP_kibolet_by_taz['pre_small_area_taz']=EMP_kibolet_by_taz['small_area']/EMP_kibolet_by_taz['taz_area']

EMP_kibolet_by_taz=EMP_kibolet_by_taz.query('(pre_small_area_taz >0.7) | (pre_small_area_emp >0.1)').drop(columns='geometry_buff')

EMP_kibolet_by_taz=EMP_kibolet_by_taz.set_index('ID')


In [63]:
EMP_kibolet_by_taz['id_area_for_pre']=EMP_kibolet_by_taz.reset_index().groupby(by='ID')['small_area'].sum()

EMP_kibolet_by_taz['prec_from_id']=EMP_kibolet_by_taz['small_area']/EMP_kibolet_by_taz['id_area_for_pre']



מתחמי תעסוקה גדולים מאוד פיזור של המועסקים מבוסס על הפיזור של גירסא 3

In [64]:

path=r'{}\background_files'.format(cwd)
pre_for_dis_emp_by_taz_v3=up_load_df(path,'pre_for_dis_emp_by_taz_v3')

In [65]:
EMP_kibolet_by_taz=EMP_kibolet_by_taz.merge(pre_for_dis_emp_by_taz_v3[['Taz_num','pre_dis_emp']],on='Taz_num',how='left').fillna(0)

In [66]:
EMP_kibolet_by_taz.loc[EMP_kibolet_by_taz['name'].isin(list(pre_for_dis_emp_by_taz_v3['name'])),'prec_from_id']=0 #לאפס את המתחמים שאני רוצה לפזר לפי גירסא 3

In [67]:

EMP_kibolet_by_taz['emp_without_palestin_in_taz']=EMP_kibolet_by_taz['emp_without_palestin']*EMP_kibolet_by_taz['prec_from_id']

In [68]:
EMP_kibolet_by_taz.loc[EMP_kibolet_by_taz['name'].isin(list(pre_for_dis_emp_by_taz_v3['name'])),'emp_without_palestin_in_taz']=EMP_kibolet_by_taz['emp_without_palestin']*EMP_kibolet_by_taz['pre_dis_emp']

In [69]:

taz['emp_not_okev']=drop_geo(EMP_kibolet_by_taz).pivot_table(index='Taz_num',aggfunc='sum')[['emp_without_palestin_in_taz']]



In [70]:
taz=taz.fillna(0)

taz['total_emp']=taz['emp_not_okev']+taz['emp_okev']+taz['emp_Education']



-89397.83203043498

In [73]:
emp_in_jtmt_factor_to_emp_in_taz=taz.query('main_secto!="Palestinian"')['total_emp'].sum()/emp_in_jtmt_area

In [81]:
emp_in_jtmt_factor_to_emp_in_taz=round(emp_in_jtmt_factor_to_emp_in_taz,2)

In [83]:
save_path=r'{}\Intermediates\emp_in_jtmt_factor_to_emp_in_taz.txt'.format(cwd)

# שמירת המספר בקובץ טקסט
with open(save_path, 'w') as f:
    f.write(str(emp_in_jtmt_factor_to_emp_in_taz))
    


## התפלגות ענפי תעסוקה

In [84]:
col=['Indus',
'Com_hotel',
'Business',
'Public',
'agri']

In [85]:
for i in col:
    taz['{}'.format(i)]=taz['{}'.format(i)]*(taz['emp_not_okev']+taz['emp_okev'])

In [86]:
taz['check_emp_sum']=taz[col].sum(axis=1)-(taz['emp_not_okev']+taz['emp_okev'])

taz.query('check_emp_sum>10 |check_emp_sum<-10')

,Taz_name,jeru_metro,in_jerusal,main_secto,SCHN_NAME,ENG_NAME_n,zonetype,Muni_Heb,Sug_Muni,CR_PNIM,highBusine,SCHOOLDIST,Urban,EIProp,poly_puma,F3,F2,F1,agri,Indus,Com_hotel,Business,Public,commuting,ieold,IEProp,perScaled,FreeBuffer,PaidBuffer,Rest_EmpBu,searchtime,walktime,cost,Agg_taz_nu,area,CITYCODE1,CITYCODE2,CITYCODE3,CITYCODE4,codeseq,codeseqCon,county,majunivenr,parktot,superZone,Taz1,slope,REGION,yosh,jerusalem_,geometry,SEA1,SEA2,SEA3,UOA1,UOA2,UOA3,ARA1,ARA2,ARA3,student_chardi_not_gov,student_arab_not_gov,TOA1,TOA2,TOA3,sector_for_chinuc,student,Univ_AR,Univ_SE,Univ_UO,emp_uni,univ,yeshiva_dorms_pop_15,yeshiva_dorms_pop_20,yeshiva_dorms_pop_25,yeshiva_dorms_pop_sum,emp_from_Yeshiva_student,Yeshiva,Seminar_dorms_pop_15,Seminar_dorms_pop_20,Seminar_dorms_pop_25,Seminar_dorms_pop_sum,emp_from_Seminar_student,Seminar,kollim_demand,add_from_kollim_demand,student_yeshiva_and_kollim,UO_Hi_Ed,emp_UO_Hi_Ed,UNIVENRORTHFEMALE,UNIVENRORTHMALE,jew,emp_from_student,emp_Education,aprt_20,pop,pop_0,pop_10,pop_15,pop_20,pop_25,pop_30,pop_35,pop_40,pop_45,pop_5,pop_50,pop_55,pop_60,pop_65,pop_70,pop_75up,hh_size,student_dorms,pop_without_dorms_yeshiva,pop_20_just_from_aprt,pop_25_just_from_aprt,pop_check,pop_15_just_from_aprt,work_age,under_work_age,over_work_age,pop_emp,pop_emp_employed,pop_emp_employed_out_of_jtmt_area,taz_area,okev,emp_okev,emp_not_okev,total_emp,check_emp_sum
Taz_num,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6004.000,פיקוד מרכז,1.000,yes,U_Orthodox,נוה יעקב,Jerusalem,Jerusalem,ירושלים,עירייה,3000,0.000,5.000,2.000,0.010,7.000,3.000,6.000,18.000,0.000,0.000,0.000,0.000,0.000,0.050,0.050,0.050,0.990,1000.000,0.000,500.000,2.000,2.000,0.000,60.000,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,"POLYGON Z ((222515.872 638921.272 0.000, 22265...",0,0,0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,U_Orthodox,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,195928.143,0,0.000,538.829,538.829,-538.829
4305.000,גן החיות התנכי,1.000,yes,Jewish,גבעת משואה,Jerusalem,Jerusalem,ירושלים,עירייה,3000,0.000,5.000,2.000,0.020,5.000,2.000,7.000,17.000,0.000,0.000,0.000,0.000,0.000,0.050,0.050,0.050,0.980,1304.000,0.000,146.000,2.000,2.000,0.000,43.000,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,"POLYGON Z ((216874.874 628672.609 0.000, 21690...",0,197,196,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,197.000,196.000,Jewish,393.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1,107.967,107.967,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,519107.479,0,0.000,433.085,541.052,-433.085


## הוספת סטודנטים אל כמות משקי בית

In [87]:
taz=taz.fillna(0)

taz['hh']=taz['aprt_20']+taz['student_dorms']/1.5

## הוספת תלמידי ישיבה (באזורים לא חרדיים) אל כמות אוכלוסיה,התפלגות גילים ומשקי בית (לכלל תלמידי הישיבה)

In [88]:
taz['hh']=taz['hh']+taz['yeshiva_dorms_pop_sum']

In [89]:
taz.loc[taz['main_secto']!="U_Orthodox",'pop']=taz['pop']+taz['yeshiva_dorms_pop_sum']

taz.loc[taz['main_secto']!="U_Orthodox",'pop_15']=taz['pop_15']+taz['yeshiva_dorms_pop_15']

taz.loc[taz['main_secto']!="U_Orthodox",'pop_20']=taz['pop_20']+taz['yeshiva_dorms_pop_20']

taz.loc[taz['main_secto']!="U_Orthodox",'pop_25']=taz['pop_25']+taz['yeshiva_dorms_pop_25']

## הוספת אוכלוסיה פלסטינאית

In [90]:
path=r'{}\arab_and_palestinian\Intermediates'.format(create_forecast_basic_folder_path)
taz_demo_pls_2020=up_load_df(path,'taz_demo_pls_2020_and_pre_growth_till_2050')[['Taz_num','pop_2020']].set_index('Taz_num')

In [91]:

taz.loc[taz['main_secto']=='Palestinian','pop']=taz['pop']+taz_demo_pls_2020['pop_2020']

In [92]:
save_taz_path=r'{}\Intermediates'.format(cwd)
taz.to_excel(r'{}\taz_before_add_geo.xlsx'.format(save_taz_path))
